Load cholec80 transforms and translate to different python module with same functionality. Also translate from yaml file to pandas pickle format

In [5]:
import sys
import copy
import pandas as pd

sys.path.append('../')

from utils import load_yaml, save_yaml

servers = load_yaml('../configs/servers.yml')
server = servers['local']

databases = load_yaml('../configs/cholec80_transforms.yml')
database = databases['databases'][0]

# files, prefix in db], transform {module, type, kwargs}, database {name, prefix}
database_df = pd.DataFrame(columns=['database', 'file', 'transforms'])

# load transforms and convert them to other module's callables
key_dict = {
    'Crop': 'Crop',
    'Resize': 'Resize'
}

for db_idx, db in enumerate(databases['databases']):

    for t_idx, old_transforms in enumerate(db['transforms']):
        transforms = []
        for old_transform in old_transforms:
            # 'Crop' -> 'crop', shape -> height, width, top_left_corner -> top, left
            # 'Resize' -> 'resize', dsize -> size
            transform = {}
            for key, value in old_transform.items():
                if key == 'Crop':
                    transform = {
                        'module': 'utils.transforms',
                        'type': 'Crop',
                        'kwargs': {
                            'top_left_corner': value['top_left_corner'],
                            'shape': value['shape'][::-1],
                            'order': 'chw'
                        }
                    }
                elif key == 'Resize':
                    transform = {
                        'module': 'torchvision.transforms',
                        'type': 'Resize',
                        'kwargs': {
                            'size': value['dsize'][::-1]
                        }
                    }
                    

                else:
                    raise ValueError('Key not known')

            transforms.append(transform)                
        
        df_row = {
            'database': database['name'], 
            'file': {'name': database['videos']['files'][t_idx], 'path': database['videos']['prefix']}, 
            'transforms': transforms
        }
        database_df = database_df.append(df_row, ignore_index=True)

database_df.to_pickle('../configs/cholec80_transforms.pkl')
print(database_df.iloc[0]['transforms'])
database_df

[{'module': 'utils.transforms', 'type': 'Crop', 'kwargs': {'top_left_corner': [58, 142], 'shape': [554, 416], 'order': 'chw'}}, {'module': 'torchvision.transforms', 'type': 'Resize', 'kwargs': {'size': [480, 640]}}]


,database,file,transforms
0,cholec80,"{'name': 'video01.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'..."
1,cholec80,"{'name': 'video02.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'..."
2,cholec80,"{'name': 'video03.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'..."
3,cholec80,"{'name': 'video04.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'..."
4,cholec80,"{'name': 'video05.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'..."
...,...,...,...
70,cholec80,"{'name': 'video17.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'..."
71,cholec80,"{'name': 'video18.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'..."
72,cholec80,"{'name': 'video19.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'..."
73,cholec80,"{'name': 'video20.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'..."
